In [ ]:
#####################################
#   Dataset - Titanic Data          #
#####################################

# Titanic Data - Contains demographics and passenger information from 891 of the 2224 passengers and crew on board 
# the Titanic. 

In [ ]:
#####################################
#            Questions              #
#####################################
# Are the following factors significiant to survival? sex, pclass, age, sibsp, parch, fare, cabin?


In [ ]:
#####################################
#          Analysize Data           #
#####################################

In [2]:
import csv as csv
import numpy as np

csv_file_object = csv.reader(open('../csv/train.csv', 'rb')) 
header = csv_file_object.next()

data = []
for row in csv_file_object:
    data.append(row)
    
data = np.array(data)

number_passengers = np.size(data[0::, 1].astype(np.float))
number_survived = np.sum(data[0::, 1].astype(np.float))
proportion_survivors = number_survived / number_passengers
print proportion_survivors



0.383838383838


In [3]:
# explore female and male survival rate
# get all female / male elements
women_only_stats = data[0::, 4] == 'female'
men_only_stats = data[0::, 4] != 'female'

women_onboard = data[women_only_stats, 1].astype(np.float)
men_onboard = data[men_only_stats, 1].astype(np.float)

proportion_women_survived = np.sum(women_onboard) / np.size(women_onboard)
proportion_men_survived = np.sum(men_onboard) / np.size(men_onboard)

print 'Proportion of women who survived is {} out of {}'.format(proportion_women_survived, np.size(women_onboard))
print 'Proportion of men who survived is {} out of {}'.format(proportion_men_survived, np.size(men_onboard))

Proportion of women who survived is 0.742038216561 out of 314
Proportion of men who survived is 0.188908145581 out of 577


In [4]:
test_file = open('../csv/test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()

test_prediction_file = open('test_genderbasemodel.csv', 'wb')
test_prediction_file_object = csv.writer(test_prediction_file)

test_prediction_file_object.writerow(["PassengerID", "Survived"])
for row in test_file_object:
    if row[3] == 'female':
        test_prediction_file_object.writerow([row[0], 1])
    else:
        test_prediction_file_object.writerow([row[0], 0])
        
test_file.close()
test_prediction_file.close()

In [40]:
#print data[0]
# explore modeling the outcome on class, gender, and ticket price
fare_ceiling = 40
data[data[0::, 9].astype(np.float) >= fare_ceiling, 9] = fare_ceiling - 1.0
fare_bracket_size = 10
number_of_price_brackets = fare_ceiling / fare_bracket_size

number_of_classes = len(np.unique(data[0::, 2]))

#print number_of_price_brackets
#print number_of_classes

# initialize survival table with zeros
survival_table = np.zeros((2, number_of_classes, number_of_price_brackets))

for i in xrange(number_of_classes):
    for j in xrange(number_of_price_brackets):
        women_only_stats = data[(data[0::,4] == 'female') & (data[0::,2].astype(np.float) == i+1) & \
                                (data[0:, 9].astype(np.float) >= j*fare_bracket_size) & \
                                (data[0:, 9].astype(np.float) < (j+1)*fare_bracket_size), 1]

        
        men_only_stats = data[(data[0::,4] != 'female') & (data[0::,2].astype(np.float) == i+1) & \
                                (data[0:, 9].astype(np.float) >= j*fare_bracket_size) & \
                                (data[0:, 9].astype(np.float) < (j+1)*fare_bracket_size), 1]
        
survival_table[0, i, j] = np.mean(women_only_stats.astype(np.float))
survival_table[1, i, j] = np.mean(men_only_stats.astype(np.float))

survival_table[ survival_table != survival_table] = 0.
print survival_table

survival_table[survival_table < 0.5] = 0
survival_table[survival_table >= 0.5] = 1
# print survival_table



[[[ 0.     0.     0.     0.   ]
  [ 0.     0.     0.     0.   ]
  [ 0.     0.     0.     0.125]]

 [[ 0.     0.     0.     0.   ]
  [ 0.     0.     0.     0.   ]
  [ 0.     0.     0.     0.24 ]]]
[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


In [39]:
# So we add a ceiling
fare_ceiling = 40
# then modify the data in the Fare column to = 39, if it is greater or equal to the ceiling
data[ data[0::,9].astype(np.float) >= fare_ceiling, 9 ] = fare_ceiling - 1.0

fare_bracket_size = 10
number_of_price_brackets = fare_ceiling / fare_bracket_size

# I know there were 1st, 2nd and 3rd classes on board
number_of_classes = 3

# But it's better practice to calculate this from the data directly
# Take the length of an array of unique values in column index 2
number_of_classes = len(np.unique(data[0::,2])) 

# Initialize the survival table with all zeros
survival_table = np.zeros((2, number_of_classes, number_of_price_brackets))

for i in xrange(number_of_classes):       #loop through each class
  for j in xrange(number_of_price_brackets):   #loop through each price bin

    women_only_stats = data[(data[0::,4] == "female")&(data[0::,2].astype(np.float)== i+1) \
                            & (data[0:,9].astype(np.float) >= j*fare_bracket_size) \
                            &(data[0:,9].astype(np.float) < (j+1)*fare_bracket_size), 1]                                                   

    men_only_stats = data[(data[0::,4] != "female")&(data[0::,2].astype(np.float)== i+1) \
                          & (data[0:,9].astype(np.float)>= j*fare_bracket_size) \
                          &(data[0:,9].astype(np.float)< (j+1)*fare_bracket_size), 1] 
    
  survival_table[0,i,j] = np.mean(women_only_stats.astype(np.float)) 
  survival_table[1,i,j] = np.mean(men_only_stats.astype(np.float))
    
survival_table[ survival_table != survival_table ] = 0.

print survival_table

survival_table[ survival_table < 0.5 ] = 0
survival_table[ survival_table >= 0.5 ] = 1 

print survial_table

test_file = open('../csv/test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()
predictions_file = open("../csv/genderclassmodel.csv", "wb")
p = csv.writer(predictions_file)
p.writerow(["PassengerId", "Survived"])

for row in test_file_object:                 # We are going to loop
                                              # through each passenger
                                              # in the test set                     
  for j in xrange(number_of_price_brackets):  # For each passenger we
                                              # loop thro each price bin
    try:                                      # Some passengers have no
                                              # Fare data so try to make
      row[8] = float(row[8])                  # a float
    except:                                   # If fails: no data, so 
      bin_fare = 3 - float(row[1])            # bin the fare according Pclass
      break                                   # Break from the loop
    if row[8] > fare_ceiling:              # If there is data see if
                                              # it is greater than fare
                                              # ceiling we set earlier
      bin_fare = number_of_price_brackets-1   # If so set to highest bin
      break                                   # And then break loop
    if row[8] >= j * fare_bracket_size\
       and row[8] < \
       (j+1) * fare_bracket_size:             # If passed these tests 
                                              # then loop through each bin 
      bin_fare = j                            # then assign index
      break 
        
        
    if row[3] == 'female':                             #If the passenger is female
        p.writerow([row[0], "%d" % \
                   int(survival_table[0, float(row[1])-1, bin_fare])])
    else:                                          #else if male
        p.writerow([row[0], "%d" % \
                   int(survival_table[1, float(row[1])-1, bin_fare])])
     
# Close out the files.
test_file.close() 
predictions_file.close()

[[[ 0.          0.          0.          0.97727273]
  [ 0.          0.          0.          1.        ]
  [ 0.          0.          0.          0.125     ]]

 [[ 0.          0.          0.          0.38372093]
  [ 0.          0.          0.          0.21428571]
  [ 0.          0.          0.          0.24      ]]]


In [ ]:
#####################################
#          Report findings          #
#####################################